# Selection Combining

_Author:_ Karl-Ludwig Besser, Technische Universität Braunschweig

This notebook is part of the paper "On Fading Channel Dependency Structures with a Positive Zero-Outage Capacity" (K.-L. Besser, P.-H. Lin, E. Jorswieck, IEEE Transactions on Communications, vol. 69, no. 10, pp. 6561-6574, Oct. 2021, [doi:10.1109/TCOMM.2021.3097755](https://doi.org/10.1109/TCOMM.2021.3097755), [arXiv:2102.02541](https://arxiv.org/abs/2102.02541)).  
If you use any of this work, please cite the above paper.

> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. This will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [ ]:
import numpy as np
from scipy import stats
from scipy import optimize
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
from selection_combining import max_zoc_sc_heterog, max_zoc_sc_homog
from maximum_ratio_combining import max_zoc_inner_bound_mrc_homog, max_zoc_outer_bound_mrc_homog
from utils import w_copula

# Rayleigh Fading

The first example is Rayleigh fading. In this case, the channel gains $|H_i|^2=X_i$ are exponentially distributed according to $X_i\sim\exp(1/\rho_i)$, where $\rho_i$ denotes the SNR of transmission link $i$.

In [ ]:
def plot_pdf_gain_rayleigh():
    x = np.linspace(0, 10)
    fig, ax = plt.subplots(1,1)
    plot = ax.plot(x, np.zeros_like(x))[0]
    ax.set_ylim([0, 1])
    ax.set_xlim([min(x), max(x)])
    ax.set_xlabel("x")
    ax.set_ylabel("PDF of the Channel Gain $\\mathbf{X}=\\rho\\left|h\\right|^2$")
    
    def update_plot(snr_db=5.):
        snr = 10**(snr_db/10.)
        dist = stats.expon(scale=snr)
        plot.set_ydata(dist.pdf(x))
        
    interact(update_plot, snr_db=(0., 10., .2))

In [ ]:
plot_pdf_gain_rayleigh()

## Two-Dimensional Heterogeneous Case

In [ ]:
def plot_heterog_rayleigh():
    a, stepsize = np.linspace(0, 5, 150, retstep=True)
    A, B = np.meshgrid(a, a)
    fig, ax = plt.subplots(1,1)
    
    def update_plot(snr_x_db=0, snr_y_db=0):
        snr_x = 10**(snr_x_db/10.)
        snr_y = 10**(snr_y_db/10.)
        rv_x = stats.expon(scale=snr_x)
        rv_y = stats.expon(scale=snr_y)
        cdf_x = rv_x.cdf(A)
        cdf_y = rv_y.cdf(B)
        joint_cdf = w_copula(cdf_x, cdf_y)
        ax.clear()
        _gradx = np.gradient(joint_cdf, stepsize, axis=0)
        joint_pdf = np.gradient(_gradx, stepsize, axis=1)
        ax.set_xlim([0, 5])
        ax.set_ylim([0, 5])
        ax.pcolormesh(A, B, joint_pdf, vmin=0, shading="auto")
        ax.plot(a, rv_y.ppf(1-rv_x.cdf(a)), 'r-')
        zoc_sc = max_zoc_sc_heterog([rv_x.ppf, rv_y.ppf])
        s_zoc_sc = 2**zoc_sc - 1
        ax.plot([0, s_zoc_sc, s_zoc_sc], [s_zoc_sc, s_zoc_sc, 0], 'w--')
        
    interact(update_plot, snr_x_db=(-5, 10, 1), snr_y_db=(-5, 10, 1))

The below plot shows the joint distribution for two countermonotonic Rayleigh fading links $X_1, X_2$ with different SNRs.

The white dashed line indicates the maximum $s_{\text{SC}}=2^R-1$ corresponding to the maximum ZOC when SC is used at the receiver.

In [ ]:
plot_heterog_rayleigh()

## N-Dimensional Homogeneous Case

In [ ]:
def plot_homog_rayleigh():
    n = np.arange(2, 21)
    fig, ax = plt.subplots(1,1)
    funcs = {"Selection Combining": max_zoc_sc_homog,
             "MRC - Inner Bound": max_zoc_inner_bound_mrc_homog,
             "MRC - Outer Bound": max_zoc_outer_bound_mrc_homog}
    plots = {k: ax.plot(n, np.zeros_like(n), 'o-', label=k)[0] for k in funcs.keys()}
    ax.legend()
    ax.set_ylim([0, 10])
    ax.set_xlabel("n")
    ax.set_ylabel("Max ZOC")
    
    def update_plot(snr_db=5.):
        snr = 10**(snr_db/10.)
        dist = stats.expon(scale=snr)
        for _name, _func in funcs.items():
            plots[_name].set_ydata(_func(dist.ppf, n))
        
    interact(update_plot, snr_db=(0., 10., .2))

For the $n$-dimensional case with homogeneous Rayleigh fading links, we can calculate the maximum ZOC when SC is used at the receiver.

In the plot below, the maximum is shown over $n$ and compared to the bounds on the maximum ZOC when MRC is used at the receiver. Details can be found in the paper and the notebook [Maximum Ratio Combining.ipynb](Maximum%20Ratio%20Combining.ipynb).

In [ ]:
plot_homog_rayleigh()

# Nakagami-m Fading

Similarly to the above [Rayleigh fading example](#Rayleigh-Fading), we provide interactive plots for Nakagami-$m$ fading below.

In this case, the channel gain $X=\rho|H|^2$ is distributed according to a Gamma distribution, $X\sim\Gamma(m, \rho/m)$.

In [ ]:
def plot_pdf_gain_nakagami():
    x = np.linspace(0, 10)
    fig, ax = plt.subplots(1,1)
    plot = ax.plot(x, np.zeros_like(x))[0]
    ax.set_ylim([0, 1])
    ax.set_xlim([min(x), max(x)])
    ax.set_xlabel("x")
    ax.set_ylabel("PDF of the Channel Gain $\\mathbf{X}=\\rho\\left|h\\right|$")
    
    def update_plot(snr_db=5., m=5):
        snr = 10**(snr_db/10.)
        dist = stats.gamma(a=m, scale=snr/m)
        plot.set_ydata(dist.pdf(x))
        
    interact(update_plot, snr_db=(0., 10., .2), m=(1, 10, 1))

In [ ]:
plot_pdf_gain_nakagami()

## Two-Dimensional Heterogeneous Case

In [ ]:
def plot_heterog_nakagami():
    a, stepsize = np.linspace(0, 5, 150, retstep=True)
    A, B = np.meshgrid(a, a)
    fig, ax = plt.subplots(1,1)
    
    def update_plot(snr_x_db=0, snr_y_db=0, m=5):
        snr_x = 10**(snr_x_db/10.)
        snr_y = 10**(snr_y_db/10.)
        rv_x = stats.gamma(a=m, scale=snr_x/m)
        rv_y = stats.gamma(a=m, scale=snr_y/m)
        cdf_x = rv_x.cdf(A)
        cdf_y = rv_y.cdf(B)
        joint_cdf = w_copula(cdf_x, cdf_y)
        ax.clear()
        _gradx = np.gradient(joint_cdf, stepsize, axis=0)
        joint_pdf = np.gradient(_gradx, stepsize, axis=1)
        ax.set_xlim([0, 5])
        ax.set_ylim([0, 5])
        ax.pcolormesh(A, B, joint_pdf, vmin=0, shading="auto")
        ax.plot(a, rv_y.ppf(1-rv_x.cdf(a)), 'r-')
        zoc_sc = max_zoc_sc_heterog([rv_x.ppf, rv_y.ppf])
        s_zoc_sc = 2**zoc_sc - 1
        ax.plot([0, s_zoc_sc, s_zoc_sc], [s_zoc_sc, s_zoc_sc, 0], 'w--')
        
    interact(update_plot, snr_x_db=(-5, 10, 1), snr_y_db=(-5, 10, 1), m=(1, 10, 1))

In [ ]:
plot_heterog_nakagami()

## N-Dimensional Homogeneous Case

In [ ]:
def plot_homog_nakagami():
    n = np.arange(2, 21)
    fig, ax = plt.subplots(1,1)
    funcs = {"Selection Combining": max_zoc_sc_homog,
             "MRC - Inner Bound": max_zoc_inner_bound_mrc_homog,
             "MRC - Outer Bound": max_zoc_outer_bound_mrc_homog}
    plots = {k: ax.plot(n, np.zeros_like(n), 'o-', label=k)[0] for k in funcs.keys()}
    ax.legend()
    ax.set_ylim([0, 10])
    ax.set_xlabel("n")
    ax.set_ylabel("Max ZOC")
    
    def update_plot(snr_db=5., m=3):
        snr = 10**(snr_db/10.)
        dist = stats.gamma(a=m, scale=snr/m)
        for _name, _func in funcs.items():
            plots[_name].set_ydata(_func(dist.ppf, n))
        
    interact(update_plot, snr_db=(0., 10., .2), m=(1, 10, 1))

In [ ]:
plot_homog_nakagami()